In [ ]:
# @title Testing Pipeline
!pip install --quiet --upgrade accelerate
!pip install git+https://github.com/huggingface/diffusers.git@main
!pip install controlnet_hinter==0.0.5
!pip install gradio

import controlnet_hinter
import gradio as gr
import torch
from diffusers import DiffusionPipeline, LCMScheduler,DDIMScheduler
from diffusers import AutoPipelineForText2Image, StableDiffusionControlNetPipeline, ControlNetModel
from diffusers.utils import load_image
from PIL import Image

# Mapping for ControlNet Model path and hinter function
CONTROLNET_MAPPING = {
    "canny_edge": {
        "model_id": "lllyasviel/sd-controlnet-canny",
        "hinter": controlnet_hinter.hint_canny
    },
    "pose": {
        "model_id": "lllyasviel/sd-controlnet-openpose",
        "hinter": controlnet_hinter.hint_openpose
    },
    "depth": {
        "model_id": "lllyasviel/sd-controlnet-depth",
        "hinter": controlnet_hinter.hint_depth
    },
    "scribble": {
        "model_id": "lllyasviel/sd-controlnet-scribble",
        "hinter": controlnet_hinter.hint_scribble,
    },
    "segmentation": {
        "model_id": "lllyasviel/sd-controlnet-seg",
        "hinter": controlnet_hinter.hint_segmentation,
    },
    "normal": {
        "model_id": "lllyasviel/sd-controlnet-normal",
        "hinter": controlnet_hinter.hint_normal,
    },
    "hed": {
        "model_id": "lllyasviel/sd-controlnet-hed",
        "hinter": controlnet_hinter.hint_hed,
    },
    "hough": {
        "model_id": "lllyasviel/sd-controlnet-mlsd",
        "hinter": controlnet_hinter.hint_hough,
    }
}
no_of_steps = 20

# how much would the prompt affect the final output.
# higher guidance scale means more preference given to the prompt.
guidace_scale = 7.0

# how much final output would follow the control image
controlnet_conditioning_scale=1.0


# Base model for Stable Diffusion
#base_model_path = "runwayml/stable-diffusion-v1-5"
base_model_path =  "digiplay/Juggernaut_final"

pipe=None
torch.cuda.empty_cache()
device = "cuda"

# ControlNet model setup
controlnet_type = "canny_edge"  # or other types depending on your requirement

# Loading the base model with ControlNet
controlnet = ControlNetModel.from_pretrained(CONTROLNET_MAPPING[controlnet_type]["model_id"], torch_dtype=torch.float16).to(device)
pipe = StableDiffusionControlNetPipeline.from_pretrained(base_model_path, controlnet=controlnet,torch_dtype=torch.float16).to(device)

# Describe the prompt for the logo
prompt = "Colorful, jungle surrounding, trees, natural, detailed, hd, 4k, best quality, extremely detailed"
negative_prompt = "nsfw, longbody, lowres, bad anatomy, bad hands, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality"




def generate_image(input_image, prompt, negative_prompt, num_steps=20, guidance_scale=7.5, controlnet_conditioning_scale=1.0):
    # Convert the uploaded image into a format suitable for ControlNet (e.g., canny edge)
    control_image = CONTROLNET_MAPPING[controlnet_type]["hinter"](input_image)

    # Perform the img2img generation
    output= pipe(
    prompt=prompt,
    width=512,
    height=512,
    negative_prompt=negative_prompt,
    image=control_image,
    controlnet_conditioning_scale=controlnet_conditioning_scale,
    num_inference_steps=no_of_steps,
    guidance_scale=guidace_scale,
)
    return output.images[0]


# Create Gradio interface
iface = gr.Interface(
    fn=generate_image,  # The function to wrap
    inputs=[
        gr.Image(type="pil", label="Upload Input Image"),  # Image upload
        gr.Textbox(label="Prompt", value="Colorful, jungle surrounding, trees, natural, detailed, hd, 4k, best quality, extremely detailed"),  # Text input for prompt
        gr.Textbox(label="Negative Prompt", value="nsfw, longbody, lowres, bad anatomy, bad hands, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality"),  # Text input for negative prompt
        gr.Dropdown(choices=list(CONTROLNET_MAPPING.keys()), value="canny_edge", label="Select ControlNet Type"),  # Dropdown for ControlNet type

    ],
    outputs=gr.Image(type="pil", label="Generated Image"),  # Output an image
    title="Image-to-Image Generator with Stable Diffusion and ControlNet",
    description="Upload an image, provide a text prompt, and adjust parameters to generate a modified image using Stable Diffusion and ControlNet.",
)

# Launch the app
iface.launch()

  Cloning https://github.com/huggingface/diffusers.git (to revision main) to /tmp/pip-req-build-dc86vnd0
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git /tmp/pip-req-build-dc86vnd0
  Resolved https://github.com/huggingface/diffusers.git to commit c96bfa5c80eca798d555a79a491043c311d0f608
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.32.0.dev0-py3-none-any.whl size=2979287 sha256=4793ee91a2e70e25adf80d545f640489b334b7de00e2c96d5107865385a41fb3
  Stored in directory: /tmp/pip-ephem-wheel-cache-n1ipegp9/wheels/2f/e1/9c/c4e22972ee698b020d29ed7cc914d898247d01e302f25df1a4
Successfully built diffusers
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.31.0
    Uninstalling diffusers-0.31.0:
      Successfully uninstalled diffusers-0.31.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/920 [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://178874e8ccc5e4024d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install diffusers torch
import torch
from diffusers import FluxPipeline
from huggingface_hub import login
# Replace 'your_token_here' with the token you copied
login(token=" ")
pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-schnell", torch_dtype=torch.bfloat16) # can replace schnell with dev
# to run on low vram GPUs (i.e. between 4 and 32 GB VRAM)
pipe.enable_sequential_cpu_offload()
pipe.vae.enable_slicing()
pipe.vae.enable_tiling()

pipe.to(torch.float16) # casting here instead of in the pipeline constructor because doing so in the constructor loads all models into CPU memory at once

prompt = "A cat holding a sign that says THANK YOU"
out = pipe(
    prompt=prompt,
    guidance_scale=0.,
    height=768,
    width=1360,
    num_inference_steps=4,
    max_sequence_length=256,
).images[0]
out.save("image.png")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


model_index.json:   0%|          | 0.00/536 [00:00<?, ?B/s]

Fetching 23 files:   0%|          | 0/23 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/246M [00:00<?, ?B/s]

text_encoder_2/config.json:   0%|          | 0.00/782 [00:00<?, ?B/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
! pip install gradio diffusers torch
import gradio as gr
import torch
from diffusers import FluxPipeline

# Initialize the model pipeline
pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-schnell", torch_dtype=torch.bfloat16)  # Replace schnell with dev if needed

# Enable optimizations for low VRAM GPUs
pipe.enable_sequential_cpu_offload()
pipe.vae.enable_slicing()
pipe.vae.enable_tiling()

pipe.to(torch.float16)  # Cast to float16 to save memory

def generate_image(prompt):
    # Generate the image based on the prompt
    out = pipe(
        prompt=prompt,
        guidance_scale=0.0,
        height=768,
        width=1360,
        num_inference_steps=4,
        max_sequence_length=256,
    ).images[0]

    # Save the image
    output_path = "image.png"
    out.save(output_path)

    return output_path

# Create Gradio interface
interface = gr.Interface(
    fn=generate_image,
    inputs="text",
    outputs="image",
    title="FluxPipeline Image Generator",
    description="Generate images using FluxPipeline from the diffusers library. Provide a prompt to create an image."
)

# Launch the interface
interface.launch()